In [1]:
dataset = "exp_density"
import sys; sys.path.insert(0, '..')
from exp_density_polynomial import *
from osmm import OSMM
from osmm import AlgMode
import pickle
import datetime

device = cpu


In [2]:
num_trials = 1
max_num_rounds = 100
compare_with_cvxpy = True
compare_with_mosek = False

In [3]:
"""
Structure of the alg_props dictionary is:
  <algorithm friendly name>: [<algorithm index>,
                              <number of rounds to run algorthm for (deprecated)>,
                              <algorithm update mode>]
"""
alg_props = {
        "r=20, M=20": [0, -1, AlgMode.LowRankQNBundle, 20, 20],
}
    
num_algs = len(alg_props)
solver = "MOSEK"

In [4]:
# Define structs:
objfs = np.zeros((num_trials, num_algs, max_num_rounds))
objfs_validation = np.zeros((num_trials, num_algs, max_num_rounds))
runtimes = np.zeros((num_trials, num_algs, max_num_rounds))
iters_takens = np.zeros((num_trials, num_algs))
damping_facs = np.ones((num_trials, num_algs, max_num_rounds))
mus = np.ones((num_trials, num_algs, max_num_rounds))
ts = np.ones((num_trials, num_algs, max_num_rounds))
Xs = np.zeros((n, num_trials, num_algs, max_num_rounds))
lower_bounds = np.ones((num_trials, num_algs, max_num_rounds)) * (-np.inf)
best_xs = np.zeros((n, num_trials, num_algs))
v_norms = np.zeros((num_trials, num_algs, max_num_rounds))
num_f_evas_line_searchs = np.zeros((num_trials, num_algs, max_num_rounds))
opt_res_norms = np.zeros((num_trials, num_algs, max_num_rounds))
q_norms = np.zeros((num_trials, num_algs, max_num_rounds))
f_grad_norms = np.zeros((num_trials, num_algs, max_num_rounds))
time_cost_details = np.zeros((6, num_trials, num_algs, max_num_rounds))

objfs_opt_cvxpy = np.zeros((num_trials,num_algs))
objfs_opt_mosek = np.zeros((num_trials,num_algs))
baseline_total_time_cost = np.zeros(num_trials)
baseline_mosek_solve_time = np.zeros(num_trials)

In [6]:
for trial_idx in range(num_trials):
    print("Trial %d/%d (dataset = %s):" % (trial_idx, num_trials - 1, dataset))
    W = generate_random_data()
    W_validation = generate_random_data()
    print("End generate data")
    if compare_with_mosek:
        t0 = time.time()
        x_opt_mosek, objf_opt = get_baseline_soln_mosek(W)
        baseline_mosek_solve_time[trial_idx] = time.time() - t0
        baseline_total_time_cost[trial_idx] = baseline_mosek_solve_time[trial_idx]
        print("opt objf value", objf_opt, "mosek total time", baseline_mosek_solve_time[trial_idx])
        objfs_opt_mosek[trial_idx, :] = objf_opt * np.ones(num_algs)
    else:
        objf_opt = None

    if compare_with_cvxpy:
        t0 = time.time()
        x_opt, objf_opt, baseline_mosek_solve_time[trial_idx] = get_baseline_soln_cvxpy(W, compare_with_all=False)
        baseline_total_time_cost[trial_idx] = time.time() - t0
        print("opt objf value", objf_opt, "CVXPY baseline total time", baseline_total_time_cost[trial_idx], 
              "CVXPY baseline solve time", baseline_mosek_solve_time[trial_idx])
        objfs_opt_cvxpy[trial_idx, :] = objf_opt * np.ones(num_algs)
        
    osmm_prob = OSMM(my_objf_torch, get_cvxpy_description, get_initial_val, W, W_validation, save_timing=True, store_x_all_iters=True)

    for alg_name in alg_props:
        print("    " + alg_name + ":")
        alg_idx = alg_props[alg_name][0]
        alg_mode = alg_props[alg_name][2]
        H_rank = alg_props[alg_name][3]
        M = alg_props[alg_name][4]
        
        osmm_prob.solve(max_num_rounds, alg_mode, r=H_rank, M=M, solver=solver, stop_early=False)

        Xs[:, trial_idx, alg_idx, :] = osmm_prob.method_results["X_iters"]
        objfs[trial_idx, alg_idx, :] = osmm_prob.method_results["objf_iters"]
        objfs_validation[trial_idx, alg_idx, :] = osmm_prob.method_results["objf_validation_iters"]
        damping_facs[trial_idx, alg_idx, :] = osmm_prob.method_results["lambd_iters"]
        mus[trial_idx, alg_idx, :] = osmm_prob.method_results["mu_iters"]
        ts[trial_idx, alg_idx, :] = osmm_prob.method_results["t_iters"]
        lower_bounds[trial_idx, alg_idx, :] = osmm_prob.method_results["lower_bound_iters"]
        runtimes[trial_idx, alg_idx, :] = osmm_prob.method_results["runtime_iters"]
        best_xs[:, trial_idx, alg_idx] = osmm_prob.method_results["x_best"]
        v_norms[trial_idx, alg_idx, :] = osmm_prob.method_results["v_norm_iters"]
        num_f_evas_line_searchs[trial_idx, alg_idx, :] = osmm_prob.method_results["num_f_evas_line_search_iters"]
        opt_res_norms[trial_idx, alg_idx, :] = osmm_prob.method_results["opt_res_iters"]
        f_grad_norms[trial_idx, alg_idx, :] = osmm_prob.method_results["f_grad_norm_iters"]
        q_norms[trial_idx, alg_idx, :] = osmm_prob.method_results["q_norm_iters"]
        time_cost_details[:, trial_idx, alg_idx, :] = osmm_prob.method_results["time_cost_detail_iters"]
        iters_taken = osmm_prob.method_results["iters_taken"]
        print("")

Trial 0/0 (dataset = exp_density):
N_0 =  10000 N= 9749
N_0 =  10000 N= 9763
End generate data
Start to solve baseline problem by MOSEK
  MOSEK + CVXPY time cost  1.0339698791503906
  MOSEK solver time cost 0.7521641254425049
  Setup time cost None
  Objective value 0.5250101543829169
  Solver status  optimal.

opt objf value 0.5250101543829169 CVXPY baseline total time 1.035161018371582 CVXPY baseline solve time 0.7521641254425049
    r=20, M=20:
Hutchinson #iters 27 rel. incr. 0.0007845546810127498 est. tr. 1.6678235636920564
lam_0 0.1191302545494326 Hutchinson time cost,  0.025369882583618164


/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/reductions/solvers/solving_chain.py:163: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


iter= 10 objf_k+1= 0.5274008027596836 L_k+1= 0.5147443485543495 lam_k+1= 0.0015668985834259488 tk= 1.0 mu_k+1 0.5277319168000003 ||G_k+1||_F= 0.9117857054625559 tau_k+1 0.002969118473878038
iter= 20 objf_k+1= 0.5250100296204812 L_k+1= 0.5250059988976286 lam_k+1= 0.0005263886120411513 tk= 0.125 mu_k+1 0.1473063013603451 ||G_k+1||_F= 1.0002800235280516 tau_k+1 0.0035734290195331407
iter= 30 objf_k+1= 0.5250097388034726 L_k+1= 0.5250100140005686 lam_k+1= 0.0009599478037538029 tk= 0.125 mu_k+1 0.2778724425299009 ||G_k+1||_F= 0.9835129576074524 tau_k+1 0.0034546347777919943
iter= 40 objf_k+1= 0.5250104083990588 L_k+1= 0.5250102878735212 lam_k+1= 0.0014188842214477122 tk= 0.0009765625 mu_k+1 0.3812123253852569 ||G_k+1||_F= 1.0208666781661575 tau_k+1 0.0037220313378214464
iter= 50 objf_k+1= 0.5250096715970398 L_k+1= 0.5250102878735212 lam_k+1= 0.0022511200083407384 tk= 0.03125 mu_k+1 0.5229839839551463 ||G_k+1||_F= 1.0978276004483363 tau_k+1 0.004304376572521971
iter= 60 objf_k+1= 0.525010235

In [7]:
print("W shape = ", W.shape)
_, N = W.shape
print("log 10 N, n = ", np.log10(N), n)

W shape =  (15, 9749)
log 10 N, n =  3.988960070390338 14


In [8]:
if not compare_with_cvxpy and not compare_with_mosek:
    baseline_objf_opt_s = None
elif not compare_with_mosek:
    baseline_objf_opt_s = objfs_opt_cvxpy
else:
    baseline_objf_opt_s = objfs_opt_mosek
print(baseline_objf_opt_s)

[[0.52501015]]


In [9]:
stuff = {}
stuff["objfs"] = objfs
stuff["objfs_validation"] = objfs_validation
stuff["lower_bounds"] = lower_bounds
stuff["runtimes"] = runtimes
stuff["opt_res_norms"] = opt_res_norms
stuff["f_grad_norms"] = f_grad_norms
stuff["q_norms"] = q_norms
stuff["damping_facs"] = damping_facs
stuff["ts"] = ts
stuff["Xs"] = Xs
stuff["best_xs"] = best_xs
stuff["v_norms"] = v_norms
stuff["num_f_evas_line_searchs"] = num_f_evas_line_searchs
stuff["iters_taken"] = iters_taken

stuff["time_cost_details"] = time_cost_details

stuff["alg_props"] = alg_props
stuff["baseline_total_time_cost"] = baseline_total_time_cost
stuff["baseline_mosek_solve_time"] = baseline_mosek_solve_time
stuff["baseline_objf_opt_s"] = baseline_objf_opt_s
now = datetime.datetime.now()
mmddyyhhmm = ("%d_%d_%d_%d_%d" % (now.month, now.day, now.year, now.hour, now.minute))
part_of_out_fn = dataset + "_pnt1_" + mmddyyhhmm

In [10]:
# pickle.dump(stuff, open("data_%s.pkl" % (part_of_out_fn), "wb"))